# INSERTION PRODUIT BDD

###### 

----------------

In [ ]:
import os
import pandas as pd
import psycopg2
import csv
import reprlib
import numpy

#### Test avec pandas et sqlalchemy

In [ ]:
excel_file = 'INVENTAIRE_A.xlsx'
inventaire = pd.read_excel(excel_file, skiprows=2)

del inventaire['Unnamed: 5']
inventaire.columns = ['NOM', 'VOLUME', 'DEGRE', 'NOMBRE', 'PRIX', 'MARQUE']
index_with_nan = inventaire.index[inventaire.isnull().any(axis=1)]
inventaire.drop(index_with_nan,0, inplace=True)
inventaire.head()

In [ ]:
# connexion à la bdd collection
try:
    conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection")
    
    cur = conn.cursor()
    sql = """INSERT INTO produit (nom, volume, degre) VALUES (%s,%s,%s)"""
    value = (inventaire['NOM'], inventaire['VOLUME'], inventaire['DEGRE'])
    cur.execute(sql, value)
    conn.commit()
    count = cur.rowcount
    print (count, "enregistrement inséré avec succès dans la table person.")
  
    #fermeture de la connexion à la base de données
    cur.close()
    conn.close()
    print("La connexion PostgreSQL est fermée")
except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de l'insertion dans la table person", error)

In [ ]:
from sqlalchemy import create_engine
import os

user = 'postgres'
password = os.environ.get('pg_psw')
host = 'localhost'
port = '5432'
dbname = 'collection'  

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

#inventaire.to_sql('rhum', engine, if_exists='replace', index=False)


In [ ]:
inventaire.to_sql('produit2', engine, if_exists='append', index=False)

In [ ]:
product_inv = inventaire[['NOM', 'VOLUME', 'DEGRE']]

-------

#### Test insertion table produit collection2

In [ ]:
import numpy

In [ ]:
product_inv = inventaire[['NOM', 'VOLUME', 'DEGRE']]

In [ ]:
tuples = [tuple(x) for x in product_inv.to_numpy()]
# Comma-separated dataframe columns
cols = ','.join(list(product_inv.columns))

In [ ]:
# connexion à la bdd collection
try:
    conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection2")
    
    cur = conn.cursor()
    table = 'produit'
    sql = """INSERT INTO produit (nom, volume, degre) VALUES (%s,%s,%s)""" 
    cur.executemany(sql, tuples)
    conn.commit()
    count = cur.rowcount
    print (count, "enregistrement inséré avec succès dans la table person.")
  
    #fermeture de la connexion à la base de données
    cur.close()
    conn.close()
    print("La connexion PostgreSQL est fermée")
except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de l'insertion dans la table person", error)

In [ ]:
print(tuples[0:10])

In [ ]:
print(cols)

-----------------

# Test insertion produit collection

In [ ]:
inventaire_produit =  inventaire[['NOM', 'VOLUME', 'DEGRE', 'MARQUE']]

In [ ]:
liste_inventaire = [list(x) for x in inventaire_produit.to_numpy()]
print(liste_inventaire[0:10])

In [ ]:
nom = liste_inventaire[0][0]
volume = liste_inventaire[0][1]
degre = liste_inventaire[0][2]
marque = liste_inventaire[0][3]

print(nom)
print(volume)
print(degre)
print(marque)

In [ ]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection2"
)
cur = conn.cursor()
for nom, volume, degre, marque in liste_inventaire:
    #print(producteur)
    #print(pays)
    cur.execute(f"""SELECT producteur.code_producteur 
        FROM producteur
        WHERE producteur.nom = '{marque}';
        """)
    
    code = cur.fetchone()[0]
    print(code)
    
    cur.execute(f"INSERT INTO produit (nom, volume, degre, code_producteur) VALUES ('{nom}', {volume}, {degre}, {code})")
print(type(cur))   
cur.close()
conn.commit()
conn.close()

In [ ]:
# prod
# je crée une liste regroupant les pays et les producteurs
liste_gro = []
with open('rum_data_new.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        liste_gro.append((row[0:2]))
liste_gro = liste_gro[1:]
print(liste_gro[0:10])

In [ ]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection2"
)
cur = conn.cursor()
for nom, marque in liste_gro:
    #print(producteur)
    #print(pays)
    cur.execute(f"""SELECT producteur.code_producteur 
        FROM producteur
        WHERE producteur.nom = '{marque}';
        """)
    code = cur.fetchone()[0]
    
    cur.execute(f"INSERT INTO produit (nom, code_producteur) VALUES ('{nom}', {code})")
print(type(cur))   
cur.close()
conn.commit()
conn.close()